# 3. Вытащить текст из аудио в формате wav.

Уточнив требования, ограничивающий возможный формат данных, получаем:

- Только аудио файлы формата .wav
- Длина аудио до 5 минут
- Языки: русский и английский

In [3]:
from pathlib import Path
from IPython.display import Audio, display

import warnings

warnings.filterwarnings("ignore")

audio_path = Path("data/audio")

# обычное произношение
eng = audio_path / "eng.wav"
rus = audio_path / "rus.wav"

# медленное произношение
eng_slow = audio_path / "eng_slow.wav"
rus_slow = audio_path / "rus_slow.wav"

# быстрое произношение
eng_fast = audio_path / "eng_fast.wav"
rus_fast = audio_path / "rus_fast.wav"

# аудио
eng_long = audio_path / "eng_long.wav"
rus_long = audio_path / "rus_long.wav"

# шум, речь, которую нельзя разобрать
noise = audio_path / "noise.wav"

#### Обычное произношение

In [4]:
display(Audio(eng))
display(Audio(rus))

#### Медленное произношение

In [31]:
display(Audio(eng_slow))
display(Audio(rus_slow))

#### Быстрое произношение

In [32]:
display(Audio(eng_fast))
display(Audio(rus_fast))

#### Длинное аудио

(Не даёт его прослушать в качестве виджета)

In [5]:
from librosa import get_duration

eng_long_mins = get_duration(path=eng_long) / 60
rus_long_mins = get_duration(path=rus_long) / 60
print(f"Eng Long {eng_long_mins:.2f} min\nRus Long {rus_long_mins:.2f} min")

Eng Long 4.84 min
Rus Long 6.15 min


#### Шум, который нельзя распознать

(Аккуратно, звук неприятный)

In [6]:
display(Audio(noise))

## Будем оценивать результаты на основе простого расстояния Левенштайна

In [7]:
from Levenshtein import distance

distance("Хочу в МТС", "Хач в МТС")

2

In [15]:
eng_ground_true = "I want to get an internship at MTS."
rus_ground_true = "Я хочу пройти на стажировку в МТС."

## Пробуем whisper

Одна из самых доступных и понятных моделей. Начать с неё точно можно)

In [9]:
import torch
import whisper

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model(name="base", device=device, in_memory=True)

# юзаем куда, если есть
print(f"{device =}")

device ='cuda'


#### Проверяем обычное произношение

In [17]:
result = model.transcribe(str(eng))["text"].strip()
print(distance(result, eng_ground_true), result)
result = model.transcribe(str(rus))["text"].strip()
print(distance(result, rus_ground_true), result)

0 I want to get an internship at MTS.
0 Я хочу пройти на стажировку в МТС.


#### Пробуем замедленное

In [18]:
result = model.transcribe(str(eng_slow))["text"].strip()
print(distance(result, eng_ground_true), result)
result = model.transcribe(str(rus_slow))["text"].strip()
print(distance(result, rus_ground_true), result)

0 I want to get an internship at MTS.
0 Я хочу пройти на стажировку в МТС.


#### Теперь ускоренное

In [19]:
result = model.transcribe(str(eng_fast))["text"].strip()
print(distance(result, eng_ground_true), result)
result = model.transcribe(str(rus_fast))["text"].strip()
print(distance(result, rus_ground_true), result)

0 I want to get an internship at MTS.
5 Я хочу пройти на стажировку VMTS.


#### Длинная обработка

Скорость обработки до 20 секунд кажется приемлемой для такой длитильности аудио.

Метрику близости применять не будем, так как нет доступа к тесктам.

In [20]:
%%time
print(model.transcribe(str(eng_long))["text"])

 Someone is lying about OpenAI. Either Sam Altman, the Board of Directors, or both. It is November 22nd, 2023, and you're watching the Code Report. Welcome back to the program. Let's first do a quick recap on what's happened over the last few days. On Friday around 3 p.m., Sam Altman joins a Google Meet, where he's fired by OpenAI's Board of Directors, including Chief Scientist and Co-Founder, Ilya Sutskiver. A few minutes later, an OpenAI statement goes public, flaning how Altman was fired for not being consistently candid in his communications with the board, which implies some sort of lying by omission. OpenAI President Greg Brockman was also demoted to Assistant to the CEO. However, he promptly quit, rather than be disrespected like this. In addition, Sam Altman sent this cryptic tweet saying, I love you all, making it obvious that Ilya had betrayed him, giving us a narrative that plays out almost exactly like Shakespeare's Julie Caesar, where the dictator becomes too much of a dic

In [21]:
%%time
print(model.transcribe(str(rus_long))["text"])

 Я, если честно и сам, занимаюсь своего рода спортивным программированием, приседаю по 500 раз между диплоями. Вот знайте короче, к нам в тилеге постоянно залитают молодые и шутливые ребята, которые хотят вкатиться в отюшечку. Ничего зазорного я в этом не вижу, ведь все мы такими были. Но вот в последнее время меня немного настраживает один факт, все чаще и чаще упоминается спортивное программирование, а конкретно в контексте решения алимпиадных задачек для прокачивания собственного скила в Кодинге. Код мысли простого рабочий Иван Город дверь, я, конечно, понимаю, ведь все крутые программисты, мягом мозги и способный решитель любой алгоритмическую задачу, затратив на нее 64 кл обает памяти, да, и их код всегда выполнится за парочку на на секунд. Исходя из этого, конечно же, напрашиваются простые выводы. Ну типа, если я буду решать такие же задачки, как и тот крутой прогер, значит я сам стану таким же. Это конечно же лезобетонная логика, но есть парочка нюансов. Да и сразу хочу предвари

#### Убеждаемся, что в шуме нет слов

In [27]:
model.transcribe(str(noise))["text"]

''

## Итог

На данную задачу, считаю простое использования whisper достаточным, русский и английский расспознаются хорошо.

Плюс, можно попробовать большую модель, на ней, результаты должны быть лучше.